1.

In [56]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import requests
import os

def download_data(url, filename):
    response = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(response.content)

url_base = 'https://www.tensorflow.org/datasets/catalog/fashion_mnist'
filenames = ['train-images-idx3-ubyte.gz', 'train-labels-idx1-ubyte.gz', 't10k-images-idx3-ubyte.gz', 't10k-labels-idx1-ubyte.gz']
for filename in filenames:
    if not os.path.exists(filename):
        download_data(url_base + filename, filename)

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

print("Розмірність навчальних зображень:", train_images.shape)
print("Розмірність навчальних міток:", train_labels.shape)
print("Розмірність тестових зображень:", test_images.shape)
print("Розмірність тестових міток:", test_labels.shape)


Розмірність навчальних зображень: (60000, 28, 28)
Розмірність навчальних міток: (60000,)
Розмірність тестових зображень: (10000, 28, 28)
Розмірність тестових міток: (10000,)


2.

In [57]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

base_model = Sequential([
    Flatten(input_shape=(28, 28)),   
    Dense(128, activation='relu'),   
    Dense(10, activation='softmax')  
])

base_model.compile(optimizer='adam',  
                   loss='categorical_crossentropy', 
                   metrics=['accuracy'])   

base_model.summary()

train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.1, random_state=42)

num_classes = 10
train_labels = to_categorical(train_labels, num_classes)
val_labels = to_categorical(val_labels, num_classes)


print("Розмірність train_labels:", train_labels.shape)
print("Розмірність val_labels:", val_labels.shape)

history = base_model.fit(train_images, train_labels, epochs=10, batch_size=128, validation_split=0.1)


Model: "sequential_163"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_163 (Flatten)           │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_467 (Dense)               │ (None, 128)            │       100,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_468 (Dense)               │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 101,770 (397.54 KB)

 Trainable params: 101,770 (397.54 KB)

 Non-trainable params: 0 (0.00 B)

Розмірність train_labels: (54000, 10)
Розмірність val_labels: (6000, 10)
Epoch 1/10
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6570 - loss: 26.8157 - val_accuracy: 0.6720 - val_loss: 1.6282
Epoch 2/10
380/380 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6934 - loss: 1.3026 - val_accuracy: 0.7461 - val_loss: 0.9591
Epoch 3/10
380/380 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7467 - loss: 0.8569 - val_accuracy: 0.7411 - val_loss: 0.7764
Epoch 4/10
380/380 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7623 - loss: 0.6783 - val_accuracy: 0.7622 - val_loss: 0.7013
Epoch 5/10
380/380 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7858 - loss: 0.5848 - val_accuracy: 0.7865 - val_loss: 0.6453
Epoch 6/10
380/380 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8116 - loss: 0.5163 - val_accuracy: 0.7950 - val_loss: 0.6022
Epoch 7/10
380/380 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8286 - loss: 0.4755 - val_accuracy: 0.8163 - val_loss: 0.5358
Epoch 8/10
380/380 ━━━━━━━━━━━

3.1 експеримент, оцінюючи моделі з різною кількістю нейронів у кожному шарі

In [58]:
def build_model(num_neurons):
    model = Sequential()
    model.add(Flatten(input_shape=(28, 28)))
    model.add(Dense(num_neurons, activation='relu'))  
    model.add(Dense(10, activation='softmax'))  
    return model

num_neurons_list = [64, 128, 256]  
for num_neurons in num_neurons_list:
    model = build_model(num_neurons)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    print(f"\nМодель з {num_neurons} нейронами у шарі:")
    model.summary()
    history = model.fit(train_images, train_labels, epochs=10, batch_size=128, validation_split=0.1)



Модель з 64 нейронами у шарі:

Model: "sequential_164"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_164 (Flatten)           │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_469 (Dense)               │ (None, 64)             │        50,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_470 (Dense)               │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 50,890 (198.79 KB)

 Trainable params: 50,890 (198.79 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_166 (Flatten)           │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_473 (Dense)               │ (None, 256)            │       200,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_474 (Dense)               │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 203,530 (795.04 KB)

 Trainable params: 203,530 (795.04 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6857 - loss: 33.3720 - val_accuracy: 0.7976 - val_loss: 2.4147
Epoch 2/10
380/380 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7935 - loss: 2.0114 - val_accuracy: 0.7859 - val_loss: 1.1665
Epoch 3/10
380/380 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7823 - loss: 0.8840 - val_accuracy: 0.7939 - val_loss: 0.7004
Epoch 4/10
380/380 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8162 - loss: 0.5691 - val_accuracy: 0.8157 - val_loss: 0.6500
Epoch 5/10
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8335 - loss: 0.4951 - val_accuracy: 0.8311 - val_loss: 0.5700
Epoch 6/10
380/380 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8492 - loss: 0.4315 - val_accuracy: 0.8298 - val_loss: 0.5516
Epoch 7/10
380/380 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8550 - loss: 0.4058 - val_accuracy: 0.8393 - val_loss: 0.5167
Epoch 8/10
380/380 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8568 - loss: 0.3988 - val_accuracy: 0

3.2 експеримент з різними функціями активації у шарах

In [59]:
def build_model(activation_func):
    model = Sequential()
    model.add(Flatten(input_shape=(28, 28)))
    model.add(Dense(128, activation=activation_func))  
    model.add(Dense(10, activation='softmax'))  
    return model

activation_functions = ['relu', 'sigmoid', 'tanh']
for activation_func in activation_functions:
    model = build_model(activation_func)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    print(f"\nМодель з функцією активації '{activation_func}':")
    model.summary()
    history = model.fit(train_images, train_labels, epochs=10, batch_size=128, validation_split=0.1)



Модель з функцією активації 'relu':


Model: "sequential_167"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_167 (Flatten)           │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_475 (Dense)               │ (None, 128)            │       100,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_476 (Dense)               │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 101,770 (397.54 KB)

 Trainable params: 101,770 (397.54 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_169 (Flatten)           │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_479 (Dense)               │ (None, 128)            │       100,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_480 (Dense)               │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 101,770 (397.54 KB)

 Trainable params: 101,770 (397.54 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6339 - loss: 1.0684 - val_accuracy: 0.7081 - val_loss: 0.7998
Epoch 2/10
380/380 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7241 - loss: 0.7793 - val_accuracy: 0.7209 - val_loss: 0.7643
Epoch 3/10
380/380 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7244 - loss: 0.7384 - val_accuracy: 0.7126 - val_loss: 0.7776
Epoch 4/10
380/380 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7185 - loss: 0.7570 - val_accuracy: 0.7154 - val_loss: 0.7574
Epoch 5/10
380/380 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7270 - loss: 0.7267 - val_accuracy: 0.7293 - val_loss: 0.7254
Epoch 6/10
380/380 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7433 - loss: 0.7106 - val_accuracy: 0.7531 - val_loss: 0.6890
Epoch 7/10
380/380 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7482 - loss: 0.6826 - val_accuracy: 0.7346 - val_loss: 0.7220
Epoch 8/10
380/380 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7169 - loss: 0.7331 - val_accuracy: 0.

3.3 експеримент з різними значеннями кількості епох та розміру батчу


In [60]:
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
from keras.utils import to_categorical

def build_model():
    model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(128, activation='relu'),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

model = build_model()

epochs_list = [5, 10, 15, 20, 25, 30] 
batch_sizes = [64, 128, 256, 512, 1024]  

for epochs in epochs_list:
    for batch_size in batch_sizes:
        print(f"\nЕксперимент з кількістю епох: {epochs}, та розміром батчу: {batch_size}")
        model = build_model() 
        history = model.fit(train_images, train_labels, epochs=epochs, batch_size=batch_size, validation_split=0.1)
        test_labels_one_hot = to_categorical(test_labels, num_classes=10)  
        test_loss, test_acc = model.evaluate(test_images, test_labels_one_hot)
        print("Test Accuracy:", test_acc)
        if test_acc >= 0.91:
            print(f"Досягнута точність більше або дорівнює 91% за {epochs} епох та розміром батчу {batch_size}")
            break  



Експеримент з кількістю епох: 5, та розміром батчу: 64
Epoch 1/5
760/760 ━━━━━━━━━━━━━━━━━━━━ 1s 986us/step - accuracy: 0.6612 - loss: 19.3655 - val_accuracy: 0.7074 - val_loss: 0.9748
Epoch 2/5
760/760 ━━━━━━━━━━━━━━━━━━━━ 1s 907us/step - accuracy: 0.7084 - loss: 0.8372 - val_accuracy: 0.7370 - val_loss: 0.7706
Epoch 3/5
760/760 ━━━━━━━━━━━━━━━━━━━━ 1s 895us/step - accuracy: 0.7627 - loss: 0.6454 - val_accuracy: 0.7935 - val_loss: 0.6157
Epoch 4/5
760/760 ━━━━━━━━━━━━━━━━━━━━ 1s 897us/step - accuracy: 0.8169 - loss: 0.5258 - val_accuracy: 0.8159 - val_loss: 0.5362
Epoch 5/5
760/760 ━━━━━━━━━━━━━━━━━━━━ 1s 886us/step - accuracy: 0.8315 - loss: 0.4758 - val_accuracy: 0.8215 - val_loss: 0.5523
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 457us/step - accuracy: 0.8179 - loss: 0.5911
Test Accuracy: 0.816100001335144

Експеримент з кількістю епох: 5, та розміром батчу: 128
Epoch 1/5
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6370 - loss: 18.5663 - val_accuracy: 0.6915 - val_loss: 1.2616
Epo

3.4 експеримент з різною кількістю шарів у моделі

In [61]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

def build_model(num_layers):
    model = Sequential()
    model.add(Flatten(input_shape=(28, 28)))
    for _ in range(num_layers):
        model.add(Dense(128, activation='relu'))  
    model.add(Dense(10, activation='softmax'))  
    return model


num_layers_list = [1, 2, 3, 4, 5, 6, 7, 8]  
for num_layers in num_layers_list:
    model = build_model(num_layers)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    print(f"\nМодель з {num_layers} шарами:")
    model.summary()
    history = model.fit(train_images, train_labels, epochs=20, batch_size=256, validation_split=0.1)




Модель з 1 шарами:


Model: "sequential_201"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_201 (Flatten)           │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_543 (Dense)               │ (None, 128)            │       100,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_544 (Dense)               │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 101,770 (397.54 KB)

 Trainable params: 101,770 (397.54 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_208 (Flatten)           │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_578 (Dense)               │ (None, 128)            │       100,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_579 (Dense)               │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_580 (Dense)               │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_581 (Dense)               │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_582 (Dense)               │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_583 (Dense)               │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_584 (Dense)               │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_585 (Dense)               │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_586 (Dense)               │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 217,354 (849.04 KB)

 Trainable params: 217,354 (849.04 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
190/190 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6322 - loss: 1.9371 - val_accuracy: 0.8289 - val_loss: 0.4915
Epoch 2/20
190/190 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8351 - loss: 0.4629 - val_accuracy: 0.8372 - val_loss: 0.4584
Epoch 3/20
190/190 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8484 - loss: 0.4151 - val_accuracy: 0.8589 - val_loss: 0.3904
Epoch 4/20
190/190 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8678 - loss: 0.3636 - val_accuracy: 0.8504 - val_loss: 0.4112
Epoch 5/20
190/190 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8723 - loss: 0.3549 - val_accuracy: 0.8654 - val_loss: 0.3842
Epoch 6/20
190/190 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8807 - loss: 0.3244 - val_accuracy: 0.8637 - val_loss: 0.3875
Epoch 7/20
190/190 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8848 - loss: 0.3139 - val_accuracy: 0.8731 - val_loss: 0.3603
Epoch 8/20
190/190 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8904 - loss: 0.2953 - val_accuracy: 0.